In [ ]:
%run mpdroid.ipynb

# Ensemble of Ensembles

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv("../WA_Fn-UseC_-HR-Employee-Attrition.csv")
df.pop('EmployeeCount')
df.pop('EmployeeNumber')
df.pop('Over18')
df.pop('StandardHours')
y=df['Attrition']
X = df
X.pop('Attrition')
from sklearn import preprocessing
le = preprocessing.LabelBinarizer()
y=le.fit_transform(y)
ind_BusinessTravel = pd.get_dummies(df['BusinessTravel'],prefix='BusinessTravel')
ind_Department = pd.get_dummies(df['Department'],prefix='Department')
ind_EducationField = pd.get_dummies(df['EducationField'],prefix='EducationField')
ind_Gender = pd.get_dummies(df['Gender'],prefix='Gender')
ind_JobRole = pd.get_dummies(df['JobRole'],prefix='JobRole')
ind_MaritalStatus = pd.get_dummies(df['MaritalStatus'],prefix='MaritalStatus')
ind_OverTime = pd.get_dummies(df['OverTime'],prefix='OverTime')
df1 =pd.concat([ind_BusinessTravel, ind_Department, ind_EducationField, ind_Gender, ind_JobRole,
                ind_MaritalStatus, ind_OverTime, df.select_dtypes(['int64'])]
               , axis=1)



In [ ]:
df1.dropna(inplace=True)

In [ ]:
df1.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1, y)

# Model Stacking

## Model 1: Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)

In [ ]:
print_training_score(tree_clf, X_train, y_train)

In [ ]:
print_test_score(tree_clf, X_test, y_test)

## Model 2: Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_clf = RandomForestClassifier()

In [ ]:
rf_clf.fit(X_train, y_train.ravel())

In [ ]:
print_training_score(rf_clf, X_train, y_train)

In [ ]:
print_test_score(rf_clf, X_test, y_test)

In [ ]:
en_en = pd.DataFrame()

In [ ]:
en_en['tree_clf'] = pd.DataFrame(tree_clf.predict_proba(X_train))[1]

In [ ]:
en_en['rf_clf'] = pd.DataFrame(rf_clf.predict_proba(X_train))[1]

In [ ]:
col_name=en_en.columns

In [ ]:
 en_en = pd.concat([en_en,pd.DataFrame(y_train).reset_index(drop=True)], axis=1)

In [ ]:
en_en

In [ ]:
tmp=list(col_name)

In [ ]:
tmp.append('ind')

In [ ]:
en_en.columns = tmp

## Meta-classifier

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
m_clf = LogisticRegression(fit_intercept=False)

In [ ]:
m_clf.fit(en_en[['tree_clf','rf_clf']], en_en['ind'])

In [ ]:
en_test = pd.DataFrame()
en_test['tree_clf'] = pd.DataFrame(tree_clf.predict_proba(X_test))[1]
en_test['rf_clf'] = pd.DataFrame(rf_clf.predict_proba(X_test))[1]
en_test['combined'] = m_clf.predict(en_test[['tree_clf','rf_clf']])


In [ ]:
col_name = en_test.columns
tmp = list(col_name)
tmp.append('ind')


In [ ]:
en_test = pd.concat([en_test, pd.DataFrame(y_test).reset_index(drop=True)], axis=1)

In [ ]:
en_test.columns = tmp

In [ ]:
pd.crosstab(en_test['combined'], en_test['ind'])

In [ ]:
accuracy_score(en_test['combined'], en_test['ind'])

In [ ]:
print(classification_report(en_test['combined'], en_test['ind']))

# Single Classifier

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
df = pd.read_csv("../WA_Fn-UseC_-HR-Employee-Attrition.csv")

In [ ]:
df.Attrition.value_counts() / df.Attrition.count()

In [ ]:
class_weight = {0: 0.84, 1: 0.16}

In [ ]:
forest = RandomForestClassifier(class_weight=class_weight)

In [ ]:
ada = AdaBoostClassifier(base_estimator=forest, n_estimators=100, learning_rate=0.5, random_state=42)

In [ ]:
ada.fit(X_train, y_train.ravel())

In [ ]:
print_training_score(ada,X_train, y_train.ravel())
print_test_score(ada, X_test, y_test.ravel())

In [ ]:
bag_clf = BaggingClassifier(
    base_estimator=ada,
    n_estimators=50,
    max_samples=1.0,
    max_features=1.0,
    bootstrap=True,
    bootstrap_features=False,
    n_jobs=-1,
    random_state=42)

In [ ]:
bag_clf.fit(X_train, y_train.ravel())

In [ ]:
print_training_score(bag_clf,X_train, y_train.ravel())
print_test_score(bag_clf, X_test, y_test.ravel())